In [5]:
Threads.nthreads()

Base.Threads

In [1]:
# Example from Julia documentation
mysum = Ref(0);

Threads.@threads for i in 1:1000
    mysum[] += 1
end

In [3]:
mysum[]

1000

In [6]:
mysum = Threads.Atomic{Int64}(0)

Base.Threads.Atomic{Int64}(0)

In [7]:
Threads.@threads for i ∈ 1:1000
    Threads.atomic_add!(mysum, 1)
end

In [8]:
mysum[]

1000

In [9]:
using Base.Threads

In [11]:
using BenchmarkTools

In [13]:
v = [rand(1_000_000) for i ∈ 1:36]

36-element Vector{Vector{Float64}}:
 [0.9157983942796802, 0.038979968727382075, 0.5590676682883007, 0.8471327266935237, 0.8310397353569929, 0.08117774349363227, 0.3923431137617194, 0.11168785395260938, 0.5278095279670166, 0.029217011769528844  …  0.4000006613142364, 0.4145154369924974, 0.021141167895470825, 0.0886355315059627, 0.8325029146679261, 0.7580048019015817, 0.6842581617920354, 0.09138234652004695, 0.5665593271827212, 0.22425934697201133]
 [0.19508372103888494, 0.7669545048530217, 0.38322570874437345, 0.7989449973733254, 0.24392817649785803, 0.0043627785831729105, 0.16404547308244632, 0.5210024438132611, 0.4299657813073873, 0.06644708142644218  …  0.12738572149887828, 0.4621877552452044, 0.7484715330862629, 0.4981129171387132, 0.9713822158877582, 0.32543270020937975, 0.9853838356386516, 0.8711100412029705, 0.6338378465631808, 0.8252198897510533]
 [0.6407883925890097, 0.7484904171346016, 0.7072196829602481, 0.05045190627717189, 0.27573699263479823, 0.4375669158422393, 0.89210635

In [14]:
function vecmean(vec)
    s = 0
    count = 0
    for i in eachindex(vec)
        s += vec[i]
        count += 1
    end
    avg = s / count
    return avg
end

vecmean (generic function with 1 method)

In [15]:
vecmean(v[1])

0.49986632132561204

In [16]:
function serial_mean(vect)
    sums = zeros(length(vect))
    for i in eachindex(vect)
        sums[i] = vecmean(vect[i])
    end
    return sums
end

serial_mean (generic function with 1 method)

In [17]:
@btime serial_mean(v)

  68.809 ms (1 allocation: 368 bytes)


36-element Vector{Float64}:
 0.49986632132561204
 0.500028128988724
 0.4998143253191473
 0.49990151304276814
 0.49943094850126685
 0.4999243186084687
 0.5002437406932655
 0.5003378037796661
 0.5002384583986953
 0.5001400892644134
 ⋮
 0.4997573157860112
 0.4995731868676817
 0.499958919993611
 0.4998479811575723
 0.49980814778324734
 0.49954773305203753
 0.4999873277918828
 0.500130863460412
 0.49940937225623805

In [18]:
function threaded_mean(vect)
    sums = zeros(length(vect))
    Threads.@threads for i in eachindex(vect)
        sums[i] = vecmean(vect[i])
    end
    return sums
end

threaded_mean (generic function with 1 method)

In [19]:
@btime threaded_mean(v)

  66.840 ms (7 allocations: 928 bytes)


36-element Vector{Float64}:
 0.49986632132561204
 0.500028128988724
 0.4998143253191473
 0.49990151304276814
 0.49943094850126685
 0.4999243186084687
 0.5002437406932655
 0.5003378037796661
 0.5002384583986953
 0.5001400892644134
 ⋮
 0.4997573157860112
 0.4995731868676817
 0.499958919993611
 0.4998479811575723
 0.49980814778324734
 0.49954773305203753
 0.4999873277918828
 0.500130863460412
 0.49940937225623805

In [20]:
function spawn_mean(vect)
    tasks = [Threads.@spawn vecmean(vect[i]) for i in 1:length(vect)]
    sums = [fetch(t) for t in tasks]
    return sums
end

spawn_mean (generic function with 1 method)

In [21]:
@btime spawn_mean(v)

  68.987 ms (220 allocations: 18.16 KiB)


36-element Vector{Float64}:
 0.49986632132561204
 0.500028128988724
 0.4998143253191473
 0.49990151304276814
 0.49943094850126685
 0.4999243186084687
 0.5002437406932655
 0.5003378037796661
 0.5002384583986953
 0.5001400892644134
 ⋮
 0.4997573157860112
 0.4995731868676817
 0.499958919993611
 0.4998479811575723
 0.49980814778324734
 0.49954773305203753
 0.4999873277918828
 0.500130863460412
 0.49940937225623805

In [22]:
v1 = rand(20_000);
v2 = [rand(1000) for i in 1:143];
v = [v1, v2...];

In [23]:
sum_serial = serial_mean(v);
sum_thread = threaded_mean(v);
sum_spawn = spawn_mean(v);

In [24]:
sum_serial == sum_thread == sum_spawn

true

In [25]:
@btime serial_mean(v);
@btime threaded_mean(v);
@btime spawn_mean(v);

  277.512 μs (1 allocation: 1.22 KiB)


  279.228 μs (7 allocations: 1.77 KiB)


  322.162 μs (868 allocations: 72.22 KiB)
